In [3]:
import os
import sagemaker
import numpy as np
from sagemaker.tensorflow import TensorFlow
from tensorflow.python.keras.preprocessing.image import load_img

ON_SAGEMAKER_NOTEBOOK = False

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()



bucket = "cats-n-dogs"
key = "data"
key_output = "output"                   # Path from the bucket's root to the dataset
train_instance_type='ml.p3.2xlarge'      # The type of EC2 instance which will be used for training
deploy_instance_type='ml.m4.xlarge'     # The type of EC2 instance which will be used for deployment
hyperparameters={
    'learning_rate': 1e-4,
    'decay': 1e-6,
    'epochs': 1, 
    'batch-size' : 64
}


# data_path = os.path.join(os.getcwd())
# train_input_path = "file://{}/TRAIN/".format(data_path)
# validation_input_path = "file://{}/VAL/".format(data_path)

estimator = TensorFlow(
  entry_point="cats_n_dogs.py",             # Your entry script
  role=role,
  framework_version="1.15.2",               # TensorFlow's version
  py_version='py3',
  hyperparameters=hyperparameters,
  train_instance_count=1,                   # "The number of GPUs instances to use"
  train_instance_type='local',
)

print("Training ...")
# estimator.fit({'training': train_input_path, 'eval': validation_input_path})











Training ...


In [5]:
data_path = os.getcwd()

local_inputs = {
    'train': 'file://{}/TRAIN'.format(data_path),
    'validation': 'file://{}/VAL'.format(data_path),
    'eval': 'file://{}/TEST'.format(data_path),
}
estimator.fit(local_inputs)


'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Creating tmpz3mv_4pk_algo-1-x4qvs_1 ... 
Attaching to tmpz3mv_4pk_algo-1-x4qvs_12mdone
algo-1-x4qvs_1  | WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
algo-1-x4qvs_1  | 
algo-1-x4qvs_1  | 2020-10-08 02:56:58,007 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-x4qvs_1  | 2020-10-08 02:56:58,013 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-x4qvs_1  | 2020-10-08 02:56:59,171 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-x4qvs_1  | 2020-10-08 02:56:59,185 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-x4qvs_1  | 2020-10-08 02:56:59,198 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-x4qvs_1  | 2020-10-08 02:56:59,207 sagemaker-containers INFO     Invoki

algo-1-x4qvs_1  | WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow_core/__init__.py:1473: The name tf.estimator.inputs is deprecated. Please use tf.compat.v1.estimator.inputs instead.
algo-1-x4qvs_1  | 
algo-1-x4qvs_1  | 2020-10-08 02:57:00,947 sagemaker_tensorflow_container.training WARNING  No model artifact is saved under path /opt/ml/model. Your training job will not save any model files to S3.
algo-1-x4qvs_1  | For details of how to construct your training script see:
algo-1-x4qvs_1  | https://sagemaker.readthedocs.io/en/stable/using_tf.html#adapting-your-local-tensorflow-script
algo-1-x4qvs_1  | 2020-10-08 02:57:00,947 sagemaker-containers INFO     Reporting training SUCCESS
tmpz3mv_4pk_algo-1-x4qvs_1 exited with code 0
Aborting on container exit...
===== Job Complete =====


In [6]:
print("Deploying ...")
predictor = estimator.deploy(initial_instance_count=1, instance_type=deploy_instance_type)

print("Predictor endpoint name : %s" % predictor.endpoint)

print("Testing endpoint ...")
cat_image = load_img("./TEST/CAT/11562.jpg", target_size=(128, 128))
cat_image_array = np.array(cat_image).reshape((1, 128, 128, 3))

print(predictor.predict({'inputs_input': cat_image_array}))


Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


Deploying ...
Attaching to tmp83sl28mg_algo-1-8dfno_1
algo-1-8dfno_1  | INFO:__main__:starting services
algo-1-8dfno_1  | Traceback (most recent call last):
algo-1-8dfno_1  |   File "/sagemaker/serve.py", line 308, in <module>
algo-1-8dfno_1  |     ServiceManager().start()
algo-1-8dfno_1  |   File "/sagemaker/serve.py", line 263, in start
algo-1-8dfno_1  |     self._tfs_config_path
algo-1-8dfno_1  |   File "/sagemaker/tfs_utils.py", line 94, in create_tfs_config
algo-1-8dfno_1  |     raise ValueError('no SavedModel bundles found!')
algo-1-8dfno_1  | ValueError: no SavedModel bundles found!
tmp83sl28mg_algo-1-8dfno_1 exited with code 1
Aborting on container exit...


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 618, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 677, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/sagemaker/local/image.py", line 623, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp83sl28mg/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Proces

RuntimeError: Giving up, endpoint didn't launch correctly